In [ ]:
#|default_exp utils

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import sys
import os
from os.path import join
import gzip
from shutil import copytree
from pathlib import Path
import datetime
import json
from urllib.parse import unquote


In [ ]:
#| export

def get_version():
    """
    Return the version of clipplot installed
    Hardcoded for now
    """
    # return pkg_resources.get_distribution("clipplot").version
    return "0.0.1"

## Time, date, and number utilities

In [ ]:
#| export

def round_floats(obj, digits=5):
    """Return 2D array obj with rounded float precision"""
    return [[round(float(j), digits) for j in i] for i in obj]

In [ ]:
#| export

def timestamp():
    """Return a string for printing the current time"""
    return str(datetime.datetime.now()) + ":"

## Path and file utilities

In [ ]:
#| export

def get_path(*args, **kwargs):
    """Return the path to a JSON file with conditional gz extension"""
    sub_dir, filename = args
    out_dir = join(kwargs["out_dir"], sub_dir) if sub_dir else kwargs["out_dir"]
    if kwargs.get("add_hash", True):
        filename += "-" + kwargs["plot_id"]
    path = join(out_dir, filename + ".json")
    return path + ".gz" if kwargs.get("gzip", False) else path

In [ ]:
#| export

def write_json(path, obj, **kwargs):
    """Write json object `obj` to disk and return the path to that file"""
    out_dir, filename = os.path.split(path)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    if kwargs.get("gzip", False):
        with gzip.GzipFile(path, "w") as out:
            out.write(json.dumps(obj, indent=4).encode(kwargs["encoding"]))
        return path
    else:
        with open(path, "w") as out:
            json.dump(obj, out, indent=4)
        return path


def read_json(path, **kwargs):
    """Read and return the json object written by the current process at `path`"""
    if kwargs.get("gzip", False):
        with gzip.GzipFile(path, "r") as f:
            return json.loads(f.read().decode(kwargs["encoding"]))
    with open(path) as f:
        return json.load(f)


### Cross-version copy tree

This is copied from `_copytree` in `nbdev.quarto`.

We put this in a separate module for one main reason:

When we import this module, we can use its `__file__` attribute (available when you import anything) to define the source for copying our web files.

In [ ]:
#|export
def copytree_agnostic(a,b):
    if sys.version_info.major >=3 and sys.version_info.minor >=8: copytree(a, b, dirs_exist_ok=True)
    else:
        from distutils.dir_util import copy_tree
        copy_tree(a, b)

In [ ]:
#|hide 
# put in a test for copytree

### Filename-related functions

In [ ]:
#| export
def clean_filename(s, **kwargs):
    """Given a string that points to a filename, return a clean filename
    
    Args:
        s (str): filename path

    Returns:
        s (str): clean file name

    Notes:
        kwargs is not used at all
    
    """
    s = unquote(os.path.basename(s))
    invalid_chars = '<>:;,"/\\|?*[]'
    for i in invalid_chars:
        s = s.replace(i, "")
    return s

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()